In [48]:
import pandas as pd
import numpy as np
#import pandas_datareader as pdr
#import datetime as dt

import plotly.express as px
#import plotly.graph_objects as go

import panel as pn
pn.extension(template='fast-list')

import hvplot.pandas
from holoviews.plotting.links import RangeToolLink

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_theme()

import os
from pathlib import Path
from dotenv import load_dotenv

import holoviews as hv
#import datetime as dt


import requests
#import json
#import csv


import bokeh.sampledata
#bokeh.sampledata.download()

%config IPCompleter.use_jedi = False



In [49]:
#api_key = 'ADE7EAXVNVBQNS0Z'
load_dotenv()
api_key = os.getenv("ALPHAVANTAGE_API_KEY")

#tickers = ['AAPL', 'GOOGL','V']
#tickers = [ 'ZM', 'GS']

#returns data from March to February

# for ticker in tickers:

#     url_month1 = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={ticker}&interval=60min&apikey={api_key}&datatype=json&slice=year1month2"

#     r_month1 = requests.get(url_month1)
#     decoded_content_month1 = r_month1.content.decode('utf-8')
#     file_name_month_1 = f'Data/data_month_1_{ticker}.csv'

#     with open(file_name_month_1, 'w') as file:
#         file.write(decoded_content_month1)

# #returns data from February to January
# for ticker in tickers:


#     url_month2 = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={ticker}&interval=60min&apikey={api_key}&datatype=json&slice=year1month3"

#     r_month2 = requests.get(url_month2)
#     decoded_content_month2 = r_month2.content.decode('utf-8')
#     file_name_month_2 = f'Data/data_month_2_{ticker}.csv'

#     with open(file_name_month_2, 'w') as file:
#         file.write(decoded_content_month2)

In [50]:
tickers = ['AAPL', 'GOOGL', 'V', 'ZM', 'GS']

In [51]:
def compute_rsi(df, period=14):
    delta = df['close'].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)

    ema_up = up.ewm(com=period -1, adjust=False).mean()
    ema_down = down.ewm(com=period-1, adjust=False).mean()

    rs = ema_up/ema_down
    df['RSI'] = 100 - (100/(1 + rs))
    df = df.dropna(inplace=True)


In [86]:
def compute_ewm(df, period=14):
    df['ewm'] = df['close'].ewm(com=period-1, ignore_na=True).mean()

In [108]:
df_data = dict()
for ticker in tickers: 

    file_name_month_1 = f'Data/data_month_1_{ticker}.csv'
    file_name_month_2 = f'Data/data_month_2_{ticker}.csv'

    df_month_1 = pd.read_csv(file_name_month_1, parse_dates=['time'], infer_datetime_format=True)
    df_month_1.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    df_month_2 = pd.read_csv(file_name_month_2, parse_dates=['time'], infer_datetime_format=True)
    df_month_2.columns = ['date', 'open', 'high', 'low', 'close', 'volume']


    df_data[ticker] = pd.concat([df_month_2, df_month_1], axis=0)
    
    df_data[ticker] = df_data[ticker].sort_values(by='date')

In [109]:
idx_na = dict()
for ticker in tickers: 
    
    compute_rsi(df_data[ticker])
    compute_ewm(df_data[ticker])
    
    df_data[ticker] = df_data[ticker].set_index('date')
    df_data[ticker] = df_data[ticker].resample('H').first().fillna(value=np.nan)
    df_data[ticker] = df_data[ticker].reset_index()
    
    idx_na[ticker]= df_data[ticker]['ewm'].isna()

In [110]:
idx_na[ticker]

{'AAPL': 0       False
 1       False
 2       False
 3       False
 4       False
         ...  
 1378    False
 1379    False
 1380    False
 1381    False
 1382    False
 Name: ewm, Length: 1383, dtype: bool,
 'GOOGL': 0       False
 1       False
 2       False
 3       False
 4       False
         ...  
 1376    False
 1377    False
 1378    False
 1379     True
 1380    False
 Name: ewm, Length: 1381, dtype: bool,
 'V': 0       False
 1       False
 2       False
 3       False
 4       False
         ...  
 1425    False
 1426    False
 1427    False
 1428    False
 1429    False
 Name: ewm, Length: 1430, dtype: bool,
 'ZM': 0       False
 1       False
 2       False
 3       False
 4       False
         ...  
 1353    False
 1354    False
 1355    False
 1356    False
 1357    False
 Name: ewm, Length: 1358, dtype: bool,
 'GS': 0       False
 1       False
 2       False
 3       False
 4       False
         ...  
 1353    False
 1354    False
 1355    False
 1356    False


In [94]:
def make_plot_rsi(ticker, period):
    
    overview = df_data[ticker].hvplot.ohlc(yaxis=None, height=200, fields={'date': 'Date'})
    volume = df_data[ticker].hvplot.step('date', 'volume', height=150, xaxis=None)
    rsi = df_data[ticker].hvplot.step('date','RSI', grid=True, xaxis=None)
    ohlc = df_data[ticker].hvplot.ohlc(ylabel='Price ($)', grid=True, xaxis=None)
    
    compute_ewm(df_data[ticker], period)
    df_data[ticker].iloc[idx_na[ticker],-1] = np.nan


    
    ewm = df_data[ticker].hvplot.line('date','ewm',xaxis=None)
    
    RangeToolLink(overview.get(0), ohlc.get(0))
    
    hline_70 = hv.HLine(70)
    hline_70.opts(
        color='red', 
        line_width=1,
    )
    
    hline_30 = hv.HLine(30)
    hline_30.opts(
        color='green',  
        line_width=1.0,
    )
    
    layout = (volume + ohlc*ewm + rsi*hline_30*hline_70 + overview).cols(1)
    
    layout.opts(merge_tools=True)
    return layout 

In [95]:
title = '## Stock Explorer'

interact = pn.interact(make_plot_rsi, ticker=tickers, period=np.arange(4,21,4))

rsi_panel = pn.Column(
    title,
    pn.Row(interact[0], interact[1]),
)


stock_dashboard = pn.Tabs(
    ("RSI", rsi_panel)
)


In [96]:
stock_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] Column
                [0] Select(name='ticker', options=['AAPL', 'GOOGL', ...], value='AAPL')
                [1] DiscreteSlider(formatter='%d', name='period', options=[4, 8, 12, 16, 20], value=4)
            [1] Row
                [0] HoloViews(Layout, name='interactive141241')